## Calculate Page Rank

- [Page Rank algorithm](https://en.wikipedia.org/wiki/PageRank) was developed by _Sergey Brin_ and _Larry Page_ (founders of Google). 

- The page rank of a particular web page indicates its relative importance within a group of web pages.

- Importance of a page is defined by the importance of all the web pages that provided an outbound link to the web page in consideration.

    _If page A is out-bounding to page B and page A has a higher relative importance, then page B also has the high importance._

- Higher the page rank, higher up it will appear in a search.

$$ PR(u) = \sum\limits_{v \in B_u} \frac{PR(v)}{L(v)}$$

_where_

- $PR(u)$: page rank of page _u_

- $B_u$: a set of pages that links to page _u_

- $PR(v)$: each page _v_ in the set $B_u$

- $L(v)$: number of links from page _v_

In [1]:
import findspark
findspark.init()
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PR-Algo").getOrCreate()

import warnings
warnings.filterwarnings("ignore")

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/25 01:49:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/25 01:49:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
# create nested lists of web pages with outbound links and initializing rank

page_links = [ 
    ["a", ["b", "c", "d"]],
    ["c", ["b"]],
    ["b", ["d", "c"]],
    ["d", ["a", "c"]]
]

page_ranks = [ 
    ["a", 1],
    ["c", 1],
    ["b", 1],
    ["d", 1]
]

page_links_rdd = spark.sparkContext.parallelize(page_links, 2)
page_rank_rdd = spark.sparkContext.parallelize(page_ranks, 2)


In [9]:
page_links_rdd.collect()

[['a', ['b', 'c', 'd']], ['c', ['b']], ['b', ['d', 'c']], ['d', ['a', 'c']]]

In [10]:
page_rank_rdd.collect()

[['a', 1], ['c', 1], ['b', 1], ['d', 1]]

In [18]:
def rank_contributions(uris: list, rank: int):
    """function to calculate contributions

    Args:
        uris (list): list of web page URIs which provide outbound links to other web pages.
        rank (int): rank of the web pages accessed through the outbound links

    Returns:
        list: contribution to all web pages
    """
    num_uris = len(uris)
    rank_cont = float(rank)/num_uris # rank contribution
    new_rank = []

    for uri in uris:
        new_rank.append((uri, rank_cont))
    
    return new_rank



In [21]:
# loop for updating page rank

num_iter = 20
s = 0.85

for i in range(num_iter):
    links_rank = page_links_rdd.join(page_rank_rdd)
    contributed_rdd = links_rank.flatMap(lambda x: rank_contributions(x[1][0], x[1][1]))
    sum_ranks = contributed_rdd.reduceByKey(lambda v1, v2: v1+v2)
    page_rank_rdd = sum_ranks.map(lambda x: (x[0], (1-s) + s*x[1]))

In [22]:
page_rank_rdd.collect()

[('a', 0.5217268024809147),
 ('b', 1.357243795127982),
 ('c', 1.2463781024360086),
 ('d', 0.8746512999550939)]